## Assignment: Segmentation and Clustering Neighbourhood in Toronto

In [36]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests
import numpy as np
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
import folium
from sklearn.cluster import KMeans

print('Libraries Imported')

Libraries Imported


In [37]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html_data = requests.get(url).text
soup = BeautifulSoup(html_data,'html5lib')

In [38]:
tag_object = soup.ta

In [39]:
table_contents=[]
table=soup.find('table')
table
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)


In [40]:
# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [41]:
df.shape

(103, 3)

In [42]:
# !conda install -c conda-forge geocoder --yes
import geocoder # import geocoder

In [43]:
lat_lng = pd.read_csv('Geospatial_Coordinates.csv')
lat_lng.rename(columns ={'Postal Code':'PostalCode'}, inplace = True)
lat_lng.head()

# initialize your variable to None
# lat_lng_coords = None
# postal_code = df['PostalCode']
# postal_code
# for  x, pin in postal_code.items():
# # loop until you get the coordinates
#     while(lat_lng_coords is None):
#       g = geocoder.arcgis('{}, Toronto, Ontario'.format(pin))
#       lat_lng_coords = g.latlng
#     # print(pin)
#     latitude = lat_lng_coords[0]
#     longitude = lat_lng_coords[1]
#     lat_lng = lat_lng.append({'Latitude':latitude,'Longitude':longitude}, ignore_index = True)
    
# lat_lng['Latitude'].value_counts()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [44]:

df = df.merge(lat_lng, on = ['PostalCode'],how = 'left')
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


### Printing the number of Boroughs and Neighbourhoods

In [45]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 15 boroughs and 103 neighborhoods.


### Using geopy library to get the latitude and longitude values of Toronto

In [46]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="tr_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### Create a map of Toronto with neighbourhoods superimposed on top

In [47]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Define Foursquare Credentials and Version

In [48]:
CLIENT_ID = 'J4I2WAJEINOGGWERVMKOIYECJDP3BL0B23524WTXGX00TYED' # your Foursquare ID
CLIENT_SECRET = 'E5YPAY4DNF2AN1HZ3KUSEJAIHJARRGPJ5T5VGFRT3CDPUNJK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: J4I2WAJEINOGGWERVMKOIYECJDP3BL0B23524WTXGX00TYED
CLIENT_SECRET:E5YPAY4DNF2AN1HZ3KUSEJAIHJARRGPJ5T5VGFRT3CDPUNJK


In [49]:
df.loc[0,'Neighborhood']

'Parkwoods'